In [1]:
from prepared_data.completed_missing_data import completed_missing_data,convert_historical_date_to_date_difference, remove_described_col_and_set_index_id, map_target
import pandas as pd
import numpy as np
from prepared_data.adding_new_features import adding_new_features

train_org = pd.read_csv('data/train.csv')
train = completed_missing_data(train_org)
train = convert_historical_date_to_date_difference(train)
train = map_target(train)
train = adding_new_features(train, number_of_history_matches=1)
train = remove_described_col_and_set_index_id(train)
train

/home/andrzej/PycharmProjects/Physionet_Challenge_2021/venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3251: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


correction col type..
percent of object with nan value:  42.20
remove coach columns..
percent of object with nan value:  18.64
remove object with more that  col 80 empty...
percent of object with nan value:  18.64
remove 9 and 10 day historii
percent of object with nan value:  15.11
remove all nan data
percent of object with nan value:  15.11


,target,is_cup,home_team_history_match_date_1,home_team_history_match_date_2,home_team_history_match_date_3,home_team_history_match_date_4,home_team_history_match_date_5,home_team_history_match_date_6,home_team_history_match_date_7,home_team_history_match_date_8,...,home_team_mean_regeneration_time,away_team_history_gol_difference_1,away_team_mean_regeneration_time,league_id_ratting,home_team_history_target_1_-1.0,home_team_history_target_1_0.0,home_team_history_target_1_1.0,away_team_history_target_1_-1.0,away_team_history_target_1_0.0,away_team_history_target_1_1.0
id,,,,,,,,,,,,,,,,,,,,,
17213498,0,1,99,71,406,98,99,69,98,115,...,99.0,5.0,104.0,6.795807,1,0,0,0,0,1
17225102,0,1,144,75,96,93,314,93,98,72,...,144.0,2.0,73.0,6.795807,1,0,0,0,0,1
17213499,0,1,144,104,66,119,290,93,169,120,...,144.0,-1.0,75.0,6.795807,0,1,0,1,0,0
17201577,1,1,80,64,98,95,97,406,100,789,...,80.0,-1.0,72.0,6.795807,0,0,1,1,0,0
17201576,-1,1,98,94,95,196,91,192,169,149,...,98.0,2.0,98.0,6.795807,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17882062,0,1,547,1344,463,165,211,168,464,208,...,547.0,3.0,547.0,8.302320,0,0,1,0,0,1
17882061,-1,1,545,1133,715,168,168,168,505,168,...,545.0,1.0,545.0,8.302320,1,0,0,0,0,1
17882060,1,1,504,1344,504,168,168,168,505,168,...,504.0,4.0,504.0,8.302320,1,0,0,0,0,1


In [2]:
import re
used = set()
columns_only_first_hist = [column for column in train.columns.values if
                           re.sub('\d', '', column) not in used and (used.add(re.sub('\d', '', column)) or True)]
print(columns_only_first_hist)

['target', 'is_cup', 'home_team_history_match_date_1', 'home_team_history_is_play_home_1', 'home_team_history_is_cup_1', 'home_team_history_goal_1', 'home_team_history_opponent_goal_1', 'home_team_history_rating_1', 'home_team_history_opponent_rating_1', 'away_team_history_match_date_1', 'away_team_history_is_play_home_1', 'away_team_history_is_cup_1', 'away_team_history_goal_1', 'away_team_history_opponent_goal_1', 'away_team_history_rating_1', 'away_team_history_opponent_rating_1', 'home_team_history_gol_difference_1', 'home_team_mean_regeneration_time', 'away_team_history_gol_difference_1', 'away_team_mean_regeneration_time', 'league_id_ratting', 'home_team_history_target_1_-1.0', 'home_team_history_target_1_0.0', 'away_team_history_target_1_-1.0', 'away_team_history_target_1_0.0']


In [3]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier


for data, name in zip([train], ['train']):
    X_train, X_test, y_train, y_test = train_test_split(data.iloc[:, 1:], data.iloc[:, 0], test_size=0.33,
                                                        random_state=42)
    pipe = make_pipeline(StandardScaler(),  RandomForestClassifier(random_state=0))

    pipe.fit(X_train, y_train)
    results = pipe.predict(X_train)
    print(f'accuracy: {pipe.score(X_test, y_test):.02f}')
    print(name, results)

accuracy: 0.48
train [-1  1 -1 ...  1  0 -1]


In [ ]:
  X_train, X_test, y_train, y_test = train_test_split(train.iloc[:, 1:], train.iloc[:, 0], test_size=0.33,
                                                        random_state=42)

for data, name in zip([train], ['train']):

    pipe = make_pipeline(StandardScaler(),  RandomForestClassifier(random_state=0))

    pipe.fit(X_train, y_train)
    results = pipe.predict(X_train)
    print(f'accuracy: {pipe.score(X_test, y_test):.02f}')
    print(name, results)


In [9]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV  # Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 1000, num = 5)]
# Number of features to consider at every split
# max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(2, 110, num = 5)]
max_depth.append(None)
# Minimum number of samples required to split a node
# min_samples_split = [2, 10]
# Minimum number of samples required at each leaf node
# min_samples_leaf = [1, 4]
# Method of selecting samples for training each tree
# bootstrap = [True, False]# Create the random grid
random_grid = {
               # 'n_estimators': n_estimators,
               # 'max_features': max_features,
               'max_depth': max_depth,
               # 'min_samples_split': min_samples_split,
               # 'min_samples_leaf': min_samples_leaf,
               # 'bootstrap': bootstrap
               }

print(random_grid)

# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation,
# search across 100 different combinations, and use all available cores
gr_search = GridSearchCV(estimator = rf, param_grid = random_grid, cv = 3, verbose=3,  n_jobs = -1)
gr_search.fit(data.iloc[:, 1:], data.iloc[:, 0])

{'max_depth': [2, 29, 56, 83, 110, None]}
Fitting 3 folds for each of 6 candidates, totalling 18 fits


GridSearchCV(cv=3, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'max_depth': [2, 29, 56, 83, 110, None]}, verbose=3)

In [10]:
gr_search.best_params_

{'max_depth': 83}

In [11]:

pipe = make_pipeline(StandardScaler(),  RandomForestClassifier(random_state=0,max_depth=80))
pipe.fit(X_train, y_train)
results = pipe.predict(X_train)
print(f'accuracy: {pipe.score(X_test, y_test):.02f}')
print(name, results)


[CV 1/3] END .......................max_depth=2;, score=0.442 total time=  11.6s
[CV 3/3] END ......................max_depth=56;, score=0.487 total time= 1.8min
accuracy: 0.48
train [-1  1 -1 ...  1  0 -1]
